Ce notebook illustre notre démarche pour classer les produits sur Rakuten en exploitant les données visuelles. 
La méthodologie s'articule autour de l'extraction de caractéristiques HOG (Histogram of Oriented Gradients) des images et de l'utilisation d'un modèle de Random Forest. Nous avons intégré un rééquilibrage des classes via RandomOverSampler pour adresser le déséquilibre des classes, suivi d'une optimisation des hyperparamètres du modèle via GridSearchCV. 
Cette approche a été choisie pour sa robustesse et son efficacité face aux défis posés par les données images complexes et le déséquilibre des classes

In [1]:
import pandas as pd
import numpy as np
from skimage import io, color, feature, transform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
import os
import mlflow

In [2]:
# Chemins vers les images et les données CSV
images_path = "../../data/raw/images/image_train"
X_csv_path = "../../data/processed/X_train_update (komla).csv"
y_csv_path = "../../data/processed/Y_train_CVw08PX (komla).csv"

# Chargement des données
X_df = pd.read_csv(X_csv_path, index_col=0)
y_df = pd.read_csv(y_csv_path, index_col=0).squeeze().map(str)
X_df["image_path"] = X_df.apply(
    lambda row: os.path.join(
        images_path, f"image_{row.imageid}_product_{row.productid}.jpg"
    ),
    axis=1,
)

In [3]:
mlflow.set_tracking_uri("../../mlruns")
mlflow.set_experiment("RF HOG")
mlflow.sklearn.autolog(log_datasets=False)

In [4]:
# Fonction pour extraire les caractéristiques HOG d'une image
def extract_hog_features(image_path):
    image = io.imread(image_path)
    image_gray = color.rgb2gray(image)
    image_resized = transform.resize(image_gray, (128, 64), anti_aliasing=True)
    hog_features = feature.hog(
        image_resized, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False
    )
    return hog_features

In [5]:
# Préparation des données
sample_X = X_df.head(50000)  # Ajustez selon vos besoins
sample_y = y_df.loc[sample_X.index]

features = np.array([extract_hog_features(path) for path in sample_X["image_path"]])
labels = sample_y.values

In [6]:
# Rééquilibrage des classes avec RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(features, labels)

In [7]:
# Division des données rééquilibrées en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

In [8]:
# Paramètres pour GridSearchCV
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
}

# Création du modèle RandomForestClassifier pour GridSearchCV
rf_model = RandomForestClassifier(random_state=42)

# Création et exécution de GridSearchCV
grid_search = GridSearchCV(
    rf_model, param_grid, cv=5, scoring="f1_weighted", n_jobs=-1, verbose=1
)
with mlflow.start_run() as run:
    print("Run id:", run.info.run_id)
    grid_search.fit(X_train, y_train)

Run id: d4a6f0783f7a441c82cea249802cde4d
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/opt/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
2024/05/05 20:57:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/05/05 20:57:36 INFO mlflow.sklearn.utils: Logging the 5 best runs, 7 runs will be omitted.


In [15]:
# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score:", grid_search.best_score_)

Meilleurs paramètres: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Meilleur score: 0.9084459391024264


In [14]:
# Prédiction et évaluation sur l'ensemble de test
y_pred = grid_search.predict(X_test)
print(f"Précision sur l'ensemble de test: {accuracy_score(y_test, y_pred):.2f}")
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

Précision sur l'ensemble de test: 0.94
Rapport de classification :
              precision    recall  f1-score   support

          10       0.92      0.97      0.94      1178
        1140       0.95      0.97      0.96      1141
        1160       0.98      0.95      0.96      1244
        1180       1.00      1.00      1.00      1180
        1280       0.84      0.80      0.82      1153
        1281       0.98      0.97      0.98      1270
        1300       0.86      0.84      0.85      1192
        1301       1.00      1.00      1.00      1218
        1302       0.95      0.96      0.96      1169
        1320       0.92      0.92      0.92      1189
        1560       0.91      0.81      0.85      1204
        1920       0.94      0.95      0.95      1171
        1940       0.99      1.00      1.00      1235
        2060       0.77      0.84      0.81      1161
        2220       1.00      1.00      1.00      1215
        2280       0.84      0.93      0.88      1165
        2403  

En adoptant cette stratégie avancée de traitement d'images et de rééquilibrage des classes, nous avons obtenu une précision de classification de 94% sur notre ensemble de test. Ces résultats démontrent l'efficacité de notre approche combinée de techniques de traitement d'images, de rééquilibrage des classes, et de modélisation par apprentissage automatique, soulignant l'impact significatif sur la classification précise des produits et l'amélioration potentielle de l'expérience utilisateur sur les plateformes e-commerce.